In [0]:
from pyspark.sql.functions import lit

In [0]:
equity_df = spark.sql("SELECT * FROM personal_finance.bronze.company_listing_info").drop("PAID_UP_VALUE").withColumn("SecurityType", lit("EQUITY"))
invit_df = spark.sql("SELECT * FROM personal_finance.bronze.invit_listing_info").drop("PAID_UP_VALUE").withColumn("SecurityType", lit("INVIT"))
etf_df = spark.sql("SELECT * FROM personal_finance.bronze.etf_listing_info").withColumn("SecurityType", lit("ETF")).drop("PAID_UP_VALUE")

In [0]:
etf_df = etf_df.withColumnRenamed("Symbol", "SYMBOL") \
    .withColumnRenamed("SecurityName", "NAME_OF_COMPANY") \
    .withColumnRenamed("Underlying", "SERIES") \
    .withColumnRenamed("DateOfListing", "DATE_OF_LISTING") \
    .withColumnRenamed("PaidUpValue", "PAID_UP_VALUE") \
    .withColumnRenamed("MarketLot", "MARKET_LOT") \
    .withColumnRenamed("ISINNumber", "ISIN_NUMBER") \
    .withColumnRenamed("FaceValue", "FACE_VALUE")

In [0]:
merged_df = equity_df.unionByName(invit_df).unionByName(etf_df)

In [0]:
pascal_case_cols = [ ''.join(word.capitalize() for word in col.split('_')) for col in merged_df.columns]
renamed_df = merged_df.toDF(*pascal_case_cols)
renamed_df = renamed_df.withColumnRenamed("IsinNumber", "ISINNumber")
display(renamed_df.limit(5))

In [0]:
renamed_df.write.mode("overwrite").saveAsTable("personal_finance.silver.company_listing_info")